In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# from prd_cluster_train_demo import load_data, preprocess_data

## set up azure experiment


In [5]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment
from azureml.core import Experiment, ComputeTarget, ScriptRunConfig

In [6]:
prd_ws = Workspace.from_config()

In [7]:
list(Environment.list(prd_ws).keys())

['prd_ml',
 'prd_ml_cluster',
 'AzureML-VowpalWabbit-8.8.0',
 'AzureML-PyTorch-1.3-CPU',
 'AzureML-Triton',
 'AzureML-responsibleai-0.17-ubuntu20.04-py38-cpu',
 'AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu-inference',
 'AzureML-mlflow-ubuntu18.04-py37-cpu-inference',
 'AzureML-minimal-ubuntu18.04-py37-cpu-inference',
 'AzureML-xgboost-0.9-ubuntu18.04-py37-cpu-inference',
 'AzureML-tensorflow-1.15-ubuntu18.04-py37-cpu-inference',
 'AzureML-sklearn-1.0-ubuntu20.04-py38-cpu',
 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11.0.3-gpu-inference',
 'AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference',
 'AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu',
 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cpu-inference',
 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu',
 'AzureML-pytorch-1.10-ubuntu18.04-py37-cpu-inference',
 'AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu',
 'AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu',
 'AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu',
 'AzureML-tensorflow-2.6-u

In [8]:
Environment.list(prd_ws)['AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu']

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04:20220329.v1\n\nENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/tensorflow-2.7\n# Create conda environment\nRUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \\\n    python=3.8 pip=20.2.4\n\n# Prepend path to AzureML conda environment\nENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH\n\n# Install pip dependencies\nRUN HOROVOD_WITH_TENSORFLOW=1 pip install 'matplotlib~=3.5.0' \\\n                                          'psutil~=5.8.0' \\\n                                          'tqdm~=4.62.0' \\\n                                          'pandas~=1.3.0' \\\n                                          'scipy~=1.7.0' \\\n                                          'numpy~=1.21.

In [9]:
azure_dataset_name ='sd3'
azure_experiment_name='prd_mlops_test'
azure_env_name = 'prd_ml_cluster'
cluster_name = 'mlops-test'

In [10]:
prd_model_name = 'azml_cluster_demo_20220414'

In [11]:
target_parameter = 'rainfall_rate_composite'
profile_features = ['air_temperature', 'relative_humidity']
single_lvl_features = ['air_pressure_at_sea_level'] 

In [12]:
prd_exp = Experiment(workspace=prd_ws, name=azure_experiment_name)
prd_exp

Name,Workspace,Report Page,Docs Page
prd_mlops_test,precip_rediagnosis,Link to Azure Machine Learning studio,Link to Documentation


Get the AzML environment (basically a conda environment) from the workspace.

In [13]:
prd_env = Environment.get(workspace=prd_ws, name=azure_env_name)
prd_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04",
        "baseImageRegistry": {
            "address": "mcr.microsoft.com",
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {},
    "inferencingStackVersion": null,
    "name": "prd_ml_cluster",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "dependencies": [
                "python=3.8",

### Load data

load the data from the script so we'renot duplicating code

In [25]:
import prd_cluster_train_demo

In [27]:
import importlib 
importlib.reload(prd_cluster_train_demo)

<module 'prd_cluster_train_demo' from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/prd-ml-pipeline/code/Users/stephen.haddad/precip_rediagnosis/model_pipeline/prd_cluster_train_demo.py'>

In [30]:
import sklearn
sklearn.__version__

'0.24.2'

In [28]:
input_data = prd_cluster_train_demo.load_data(
    prd_ws,
    dataset_name=azure_dataset_name
)
data_splits, data_dims = prd_cluster_train_demo.preprocess_data(
    input_data,
    feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
)


['air_temperature_5.0', 'air_temperature_10.0', 'air_temperature_20.0', 'air_temperature_30.0', 'air_temperature_50.0', 'air_temperature_75.0', 'air_temperature_100.0', 'air_temperature_150.0', 'air_temperature_200.0', 'air_temperature_250.0', 'air_temperature_300.0', 'air_temperature_400.0', 'air_temperature_500.0', 'air_temperature_600.0', 'air_temperature_700.0', 'air_temperature_800.0', 'air_temperature_1000.0', 'air_temperature_1250.0', 'air_temperature_1500.0', 'air_temperature_1750.0', 'air_temperature_2000.0', 'air_temperature_2250.0', 'air_temperature_2500.0', 'air_temperature_2750.0', 'air_temperature_3000.0', 'air_temperature_3250.0', 'air_temperature_3500.0', 'air_temperature_3750.0', 'air_temperature_4000.0', 'air_temperature_4500.0', 'air_temperature_5000.0', 'air_temperature_5500.0', 'air_temperature_6000.0', 'relative_humidity_5.0', 'relative_humidity_10.0', 'relative_humidity_20.0', 'relative_humidity_30.0', 'relative_humidity_50.0', 'relative_humidity_75.0', 'relative

In [15]:
# these are example calls to the code for easier debugging than running on a separate cluster
# model = prd_cluster_train_demo.build_model(**data_dims)
# model = prd_cluster_train_demo.train_model(model, data_splits)

In [16]:
import datetime
log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

### Execute our training run on a cluster

In [17]:
prd_demo_compute_target = ComputeTarget(workspace=prd_ws, name=cluster_name)
prd_demo_compute_target

AmlCompute(workspace=Workspace.create(name='precip_rediagnosis', subscription_id='07efdc52-cd27-48ed-9443-3aad2b6b777b', resource_group='precip_rediagnosis'), name=mlops-test, id=/subscriptions/07efdc52-cd27-48ed-9443-3aad2b6b777b/resourceGroups/precip_rediagnosis/providers/Microsoft.MachineLearningServices/workspaces/precip_rediagnosis/computes/mlops-test, type=AmlCompute, provisioning_state=Succeeded, location=uksouth, tags={})

In [37]:
prd_demo_args = ['--dataset-name', azure_dataset_name,
                 '--target-parameter', target_parameter,
                 '--model-name', prd_model_name,
                ]

prd_demo_args += ['--profile-features']
prd_demo_args += profile_features
prd_demo_args += ['--single-level_features']
prd_demo_args += single_lvl_features

prd_demo_args

['--dataset-name',
 'sd3',
 '--target-parameter',
 'rainfall_rate_composite',
 '--model-name',
 'azml_cluster_demo_20220414',
 '--profile-features',
 'air_temperature',
 'relative_humidity',
 '--single-level_features',
 'air_pressure_at_sea_level']

In [50]:
prd_run_src = ScriptRunConfig(source_directory=os.getcwd(),
                      script='prd_cluster_train_demo.py',
                      arguments=prd_demo_args,
                      compute_target=prd_demo_compute_target,
                      environment=prd_env)

In [51]:
prd_run = prd_exp.submit(prd_run_src)
prd_run

Experiment,Id,Type,Status,Details Page,Docs Page
prd_mlops_test,prd_mlops_test_1650450496_a9ff55aa,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


## Evaluation

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(training_hist_df.epoch, training_hist_df.loss, label='training')
plt.plot(training_hist_df.epoch, training_hist_df.val_loss, c='g', label='validation')
plt.legend()
plt.ylabel('MAE [mm of precipitation]')
plt.xlabel('epochs')

In [ ]:
fig1 = plt.figure(figsize=(10, 8))
ax1 = fig1.add_subplot(1,1,1)
ax1.scatter(y_test, y_pred, s=200, c='darkblue')
ax1.plot([0, 300], [0, 300], ls="--", c=".3")
ax1.set_xlabel('Actual 3hr precip accumulation value')
ax1.set_ylabel('Predicted 3hr precip_accumulation value')

In [ ]:
prd_run.log_image(name='actual_vs_pred', plot=fig1, description='predicted vs actual 3hr accumulations of rainfall')

In [ ]:
prd_run.complete()